In [6]:
import sys
sys.path.insert(0, '..')
from modules import constants
from modules.transform import rescale_target
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np
import pickle
import os

In [11]:
X_train = pd.read_feather(os.path.join(constants.prepared_data_path, 'X_train.arrow')).to_numpy()
y_train = pd.read_feather(os.path.join(constants.prepared_data_path, 'y_train.arrow')).to_numpy()
X_val = pd.read_feather(os.path.join(constants.prepared_data_path, 'X_val.arrow')).to_numpy()
y_val = pd.read_feather(os.path.join(constants.prepared_data_path, 'y_val.arrow')).to_numpy()
X_test = pd.read_feather(os.path.join(constants.prepared_data_path, 'X_test.arrow')).to_numpy()

In [3]:
X_train

array([[ 0.87251052,  1.29027353,  1.45872789, ..., -1.23881553,
        -1.22639268, -1.16950105],
       [-1.1933868 , -0.53469248, -0.63193735, ...,  0.34369059,
         0.19677603, -0.2411225 ],
       [ 1.0589131 ,  0.58267559,  1.30484246, ..., -0.36761803,
         0.09940476,  0.46111109],
       ...,
       [ 1.62378811, -0.02520859,  0.85302308, ...,  1.60388929,
         0.81912258, -0.31346444],
       [ 0.23646648, -0.79601786, -0.88832587, ...,  0.07761504,
        -0.64380842, -1.13579324],
       [ 0.48084437, -0.32866994, -0.36884201, ..., -0.05986986,
        -0.07400783,  0.05752608]])

In [4]:
y_train

array([[ 0.2586569 , -0.00893703, -0.00676055, -0.01385412, -0.00925393,
        -0.01049532],
       [-0.95756498, -0.0078952 , -0.00676145, -0.01403727, -0.00875767,
        -0.01265333],
       [ 1.33771648, -0.00936902, -0.00675258, -0.00813822, -0.00864576,
        -0.00724459],
       ...,
       [ 0.1392927 , -0.00865594, -0.0067611 , -0.0138096 , -0.00902291,
        -0.00783696],
       [-0.13241961, -0.00917243, -0.00676156, -0.01403931, -0.00862569,
        -0.0140331 ],
       [-1.15461216, -0.00973613, -0.00675569, -0.01402509, -0.00728107,
        -0.01360902]])

In [10]:
lm = LinearRegression()
lm.fit(X_train, y_train)
print(r2_score(y_val, lm.predict(X_val)))
print(r2_score(rescale_target(y_val), rescale_target(lm.predict(X_val))))



0.02285955573956953
0.02285955573956962


array([[ 5.33869705e-01,  4.68667010e+00,  9.32206505e+04,
        -5.93048676e+03,  6.38451134e+01,  3.63947407e+03],
       [ 4.57137965e-01,  3.38168426e+02, -7.00974917e+04,
         2.56858161e+03, -9.68947301e+01, -1.07369305e+04],
       [ 4.77965550e-01,  7.91966820e+02, -1.05396758e+05,
        -1.08763709e+04, -7.27118884e+01,  5.91109083e+04],
       ...,
       [ 4.39575611e-01, -1.22997411e+02,  8.67118826e+03,
        -7.60324134e+03,  9.43980166e+01,  9.33037350e+03],
       [ 4.32876598e-01, -1.27759852e+02, -8.94151119e+03,
         2.06848160e+03, -1.82639137e+01,  3.31968636e+03],
       [ 5.72639246e-01,  5.13638659e+02,  1.09907164e+05,
         2.62741993e+04, -3.95826819e+00, -2.99295568e+04]])

In [30]:
test_preds = rescale_target(lm.predict(X_test))
test_df = pd.DataFrame(test_preds, columns=['X4', 'X11', 'X18', 'X26', 'X50', 'X3112'])
test_df.insert(0, 'id', pd.read_csv(constants.test_metadata_path).id)
test_df.to_csv('../submission.csv', index=False)

In [33]:
 pd.read_csv(constants.train_metadata_path)

,id,WORLDCLIM_BIO1_annual_mean_temperature,WORLDCLIM_BIO12_annual_precipitation,WORLDCLIM_BIO13.BIO14_delta_precipitation_of_wettest_and_dryest_month,WORLDCLIM_BIO15_precipitation_seasonality,WORLDCLIM_BIO4_temperature_seasonality,WORLDCLIM_BIO7_temperature_annual_range,SOIL_bdod_0.5cm_mean_0.01_deg,SOIL_bdod_100.200cm_mean_0.01_deg,SOIL_bdod_15.30cm_mean_0.01_deg,...,X18_mean,X26_mean,X50_mean,X3112_mean,X4_sd,X11_sd,X18_sd,X26_sd,X50_sd,X3112_sd
0,192027691,12.235703,374.466675,62.524445,72.256844,773.592041,33.277779,125,149,136,...,0.117484,1.243779,1.849375,50.216034,0.008921,1.601473,0.025441,0.153608,0.279610,15.045054
1,195542235,17.270555,90.239998,10.351111,38.220940,859.193298,40.009777,124,144,138,...,0.389315,0.642940,1.353468,574.098472,0.003102,0.258078,0.000866,0.034630,0.010165,11.004477
2,196639184,14.254504,902.071411,49.642857,17.873655,387.977753,22.807142,107,133,119,...,8.552908,0.395241,2.343153,1130.096731,NaN,NaN,NaN,NaN,NaN,NaN
3,195728812,18.680834,1473.933350,163.100006,45.009758,381.053986,20.436666,120,131,125,...,1.083629,0.154200,1.155308,1042.686546,0.011692,2.818356,0.110673,0.011334,0.229224,141.857187
4,195251545,0.673204,530.088867,50.857777,38.230709,1323.526855,45.891998,91,146,120,...,0.657585,10.919966,2.246226,2386.467180,0.006157,1.128000,0.026996,0.553815,0.107092,87.146899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55484,190558785,19.472172,244.795914,39.127552,67.074493,472.710358,27.758673,118,140,131,...,0.233690,1.783193,1.608341,969.547831,NaN,NaN,NaN,NaN,NaN,NaN
55485,194523231,13.724150,1450.000000,162.260208,43.139324,652.716858,26.694387,125,144,135,...,1.017099,12.713048,2.418300,1630.015480,0.005474,0.128133,0.117010,3.164520,0.082212,136.503697
55486,195888987,14.741204,581.866638,109.231110,89.272148,507.273010,26.874668,118,155,136,...,2.717395,10.206478,2.722599,602.229880,0.019727,0.215040,0.156309,0.919139,0.079395,26.159626
55487,135487319,16.094763,1180.838135,80.176193,22.909716,342.184021,17.346189,109,130,117,...,4.429659,9.372170,3.251739,244.387170,NaN,NaN,NaN,NaN,NaN,NaN
